In [1]:
import numpy as np
import pandas as pd
import cv2
import random
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mostCommon import most_common
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math
from sklearn import preprocessing

In [2]:
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

In [3]:
X_train=[]
X_test=[]
y_train=[]
y_test=[]

In [4]:
df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img = cv2.equalizeHist(img)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)
X=np.asarray(images_data)
y = np.asarray(my_data['Label'])

number = preprocessing.LabelEncoder()
y=number.fit_transform(y)

In [5]:
print('Applying pca to the data')
scaler = StandardScaler()
# Fit on training set only
scaler.fit(X)
# Apply transform on both training and test set
X = scaler.transform(X)
pca = PCA(n_components=270)
pca.fit(X)
X = pca.transform(X)

Applying pca to the data


In [6]:
l=0
ll=[0,1,2,3,4]
ll=[t+l for t in ll]
print(ll)
X_test=np.take(X,ll,axis=0)
y_test=np.take(y,ll)
X_train=np.delete(X,ll,axis=0)
y_train=np.delete(y,ll)
print(len(X_test))
print(len(y_test))
print(len(X_train))
print(len(y_train))

[0, 1, 2, 3, 4]
5
5
605
605


In [14]:
r = []
conf_matricies = []
f1_reports = []
f1_micro = []
total_actual=[]
total_predicted=[]
l=5
ll=[-5,-4,-3,-2,-1]
for o in range(122):
    print('---------------------------- Patient ' + str(o) + ' ------------------------------ \n')
    ll=[t+l for t in ll]
    print('Indexes: '+str(ll))
    X_test=np.take(X,ll,axis=0)
    y_test=np.take(y,ll)
    X_train=np.delete(X,ll,axis=0)
    y_train=np.delete(y,ll)
    r=[]
    r.append(random.randint(0, 1000))
    print('Random seeds are: '+str(r))
    number_of_models = 1
    Xandy = []
    models = []
    # creating models
    for i in range(number_of_models):
        models.append(LogisticRegression(random_state=r[0],solver='lbfgs', max_iter=99999999))

    # Creating array test splits for models
    # X_train, X_test, y_train, y_test
    for i in range(number_of_models):
        #append x train x test y train ytest to x
        x=[]
        x.append(X_train)
        x.append(X_test)
        x.append(y_train)
        x.append(y_test)
        Xandy.append(x)
    '''
    print('Equalizing Data...')
    for i in range(number_of_models):
        x = Xandy[i][0]
        y1 = Xandy[i][2]
        cov = np.count_nonzero(y1 == 0)
        pneu = np.count_nonzero(y1 == 1)
        reg = np.count_nonzero(y1 == 2)
        # deleting random pneumonia elements from each model set
        while cov != pneu:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 1:
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                pneu -= 1
        # deleting random regular elements from each model set
        while cov != reg:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 2:
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                reg -= 1
        Xandy[i][0] = x
        Xandy[i][2] = y1
    '''
    print('Training Models...')
    for i in range(number_of_models):
        models[i].fit(Xandy[i][0], Xandy[i][2])

    print('getting predictions of each model...')
    predicted = []
    for i in range(number_of_models):
        predicted.append(models[i].predict(Xandy[i][1]))

    voted_prediction = []
    for i in range(len(predicted[0])):
        voting = []
        for j in range(len(predicted)):
            voting.append(predicted[j][i])
        voted_prediction.append(most_common(voting))
    total_actual.append(Xandy[0][3])
    total_predicted.append(voted_prediction)
    print(voted_prediction)
    print(Xandy[0][3])
#print('Saving models....')
#np.save('models', models)
print('All Done :)')

---------------------------- Patient 0 ------------------------------ 

Indexes: [0, 1, 2, 3, 4]
Random seeds are: [743]
Training Models...
getting predictions of each model...
[1, 1, 1, 1, 1]
[0 0 0 0 0]
---------------------------- Patient 1 ------------------------------ 

Indexes: [5, 6, 7, 8, 9]
Random seeds are: [624]
Training Models...
getting predictions of each model...
[2, 2, 0, 0, 0]
[0 0 0 0 0]
---------------------------- Patient 2 ------------------------------ 

Indexes: [10, 11, 12, 13, 14]
Random seeds are: [864]
Training Models...
getting predictions of each model...
[0, 0, 0, 0, 0]
[0 0 0 0 0]
---------------------------- Patient 3 ------------------------------ 

Indexes: [15, 16, 17, 18, 19]
Random seeds are: [926]
Training Models...
getting predictions of each model...
[2, 2, 2, 2, 2]
[0 0 0 0 0]
---------------------------- Patient 4 ------------------------------ 

Indexes: [20, 21, 22, 23, 24]
Random seeds are: [505]
Training Models...
getting predictions of ea

getting predictions of each model...
[1, 1, 1, 1, 1]
[1 1 1 1 1]
---------------------------- Patient 39 ------------------------------ 

Indexes: [195, 196, 197, 198, 199]
Random seeds are: [612]
Training Models...
getting predictions of each model...
[1, 1, 1, 1, 1]
[1 1 1 1 1]
---------------------------- Patient 40 ------------------------------ 

Indexes: [200, 201, 202, 203, 204]
Random seeds are: [839]
Training Models...
getting predictions of each model...
[1, 1, 1, 1, 1]
[1 1 1 1 1]
---------------------------- Patient 41 ------------------------------ 

Indexes: [205, 206, 207, 208, 209]
Random seeds are: [846]
Training Models...
getting predictions of each model...
[2, 0, 2, 0, 2]
[1 1 1 1 1]
---------------------------- Patient 42 ------------------------------ 

Indexes: [210, 211, 212, 213, 214]
Random seeds are: [413]
Training Models...
getting predictions of each model...
[1, 1, 2, 1, 1]
[1 1 1 1 1]
---------------------------- Patient 43 ------------------------------ 

getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 77 ------------------------------ 

Indexes: [385, 386, 387, 388, 389]
Random seeds are: [719]
Training Models...
getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 78 ------------------------------ 

Indexes: [390, 391, 392, 393, 394]
Random seeds are: [341]
Training Models...
getting predictions of each model...
[1, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 79 ------------------------------ 

Indexes: [395, 396, 397, 398, 399]
Random seeds are: [711]
Training Models...
getting predictions of each model...
[2, 2, 1, 1, 2]
[2 2 2 2 2]
---------------------------- Patient 80 ------------------------------ 

Indexes: [400, 401, 402, 403, 404]
Random seeds are: [572]
Training Models...
getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 81 ------------------------------ 

getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 115 ------------------------------ 

Indexes: [575, 576, 577, 578, 579]
Random seeds are: [395]
Training Models...
getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 116 ------------------------------ 

Indexes: [580, 581, 582, 583, 584]
Random seeds are: [207]
Training Models...
getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 117 ------------------------------ 

Indexes: [585, 586, 587, 588, 589]
Random seeds are: [152]
Training Models...
getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 118 ------------------------------ 

Indexes: [590, 591, 592, 593, 594]
Random seeds are: [899]
Training Models...
getting predictions of each model...
[2, 2, 2, 2, 2]
[2 2 2 2 2]
---------------------------- Patient 119 --------------------------

In [15]:
patients_p=[]
actual=[]
for i in range(len(total_predicted)):
    patients_p.append(most_common(total_predicted[i]))
    actual.append(most_common(total_actual[i]))
i=0
j=0
correct=0
while i < len(total_actual):
    if patients_p[i]==actual[i]:
        correct+=1
    i+=1
correct

86

In [16]:
#Patient confusion matrix and frames confusion matrix
print('------------------Patient Level confusion matrix and report-------------------')
print(confusion_matrix(patients_p,actual))
print(metrics.classification_report(patients_p, actual, digits=3))

------------------Patient Level confusion matrix and report-------------------
[[19  3  6]
 [ 5 24  7]
 [ 6  9 43]]
              precision    recall  f1-score   support

           0      0.633     0.679     0.655        28
           1      0.667     0.667     0.667        36
           2      0.768     0.741     0.754        58

    accuracy                          0.705       122
   macro avg      0.689     0.696     0.692       122
weighted avg      0.707     0.705     0.706       122

